In [21]:
import pandas as pd
import jellyfish as jf

In [22]:
rs = pd.read_table('SF-Rest-SmallSample.tsv', header=0)

In [23]:
rs.shape

(50630, 15)

In [345]:
zips = rs.business_postal_code.unique()

In [348]:
for z in zips:
    c
    print(z, pc_df.shape)

94109 (4024, 15)
94110 (5757, 15)
94103 (4435, 15)
94108 (1774, 15)
nan (0, 15)
94107 (1979, 15)
94134 (896, 15)
94114 (1655, 15)
94118 (2220, 15)
94121 (1725, 15)
94111 (1431, 15)
94122 (2149, 15)
94133 (4343, 15)
94117 (1202, 15)
94104 (972, 15)
94123 (1580, 15)
94102 (3612, 15)
94105 (1349, 15)
94112 (2295, 15)
94115 (1437, 15)
94143 (28, 15)
94124 (1035, 15)
94132 (1219, 15)
94127 (606, 15)
94131 (577, 15)
94130 (24, 15)
94116 (781, 15)
94158 (90, 15)
94602 (2, 15)
94013 (9, 15)
94014 (23, 15)
941033148 (5, 15)
94101 (28, 15)
94301 (3, 15)
94120 (5, 15)
95105 (2, 15)
94544 (2, 15)
94901 (1, 15)
Ca (12, 15)
00000 (15, 15)
CA (5, 15)
941 (14, 15)
94129 (5, 15)
941102019 (6, 15)
94080 (4, 15)
92672 (8, 15)
94188 (2, 15)


In [357]:
z = rs[rs['business_postal_code'] == '94110']

In [360]:
len(z)

5757

In [359]:
z.sort_values('business_name')

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
6378,18 reasons,3674 18th st,San Francisco,CA,94110,37.761498,-122.425402,NaN,64717_20180228,2017-02-28,92.0,Routine - Unscheduled,64717_20180228_103131,Moderate risk vermin infestation,Moderate Risk
33510,18 reasons,3674 18th st,San Francisco,CA,94110,37.761498,-122.425402,NaN,64717_20151019,2017-10-19,98.0,Routine - Unscheduled,64717_20151019_103161,Low risk vermin infestation,Low Risk
5714,18 reasons,3674 18th st,San Francisco,CA,94110,37.761498,-122.425402,NaN,64717_20180228,2017-02-28,92.0,Routine - Unscheduled,64717_20180228_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk
18202,18th street commissary,3334 18th st,San Francisco,CA,94110,37.761965,-122.417948,NaN,27350_20171017,2017-10-17,83.0,Routine - Unscheduled,27350_20171017_103131,Moderate risk vermin infestation,Moderate Risk
19619,18th street commissary,3334 18th st,San Francisco,CA,94110,37.761965,-122.417948,NaN,27350_20171017,2017-10-17,83.0,Routine - Unscheduled,27350_20171017_103154,Unclean or degraded floors walls or ceilings,Low Risk
22179,18th street commissary,3334 18th st,San Francisco,CA,94110,37.761965,-122.417948,NaN,27350_20160926,2017-09-26,78.0,Routine - Unscheduled,27350_20160926_103139,Improper food storage,Low Risk
22920,18th street commissary,3334 18th st,San Francisco,CA,94110,37.761965,-122.417948,NaN,27350_20160926,2017-09-26,78.0,Routine - Unscheduled,27350_20160926_103118,Employee eating or smoking,Moderate Risk
23409,18th street commissary,3334 18th st,San Francisco,CA,94110,37.761965,-122.417948,NaN,27350_20160926,2017-09-26,78.0,Routine - Unscheduled,27350_20160926_103149,Wiping cloths not clean or properly stored or ...,Low Risk
21786,18th street commissary,3334 18th st,San Francisco,CA,94110,37.761965,-122.417948,NaN,27350_20171017,2017-10-17,83.0,Routine - Unscheduled,27350_20171017_103132,Improper thawing methods,Moderate Risk
22765,18th street commissary,3334 18th st,San Francisco,CA,94110,37.761965,-122.417948,NaN,27350_20160926,2017-09-26,78.0,Routine - Unscheduled,27350_20160926_103132,Improper thawing methods,Moderate Risk


In [365]:
x = z.drop_duplicates(subset=['business_name', 'business_address'])
len(x)

562

In [24]:
rs.business_address = rs.business_address.str.lower()
rs.business_name = rs.business_name.str.lower()

In [25]:
small = rs.head(100)

In [26]:
rs.style
rs.head(50)

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,dar bar pakistani/indian cusine,1412 polk st,San Francisco,CA,94109,37.789784,-122.420455,NaN,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
1,dragon city bakery & cafe,2367 mission st,San Francisco,CA,94110,37.759174,-122.419066,1.415583e+10,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
2,deli 23,2449 23rd st,San Francisco,CA,94110,NaN,NaN,NaN,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
3,l'acajou bakery and cafe,498 09th st ste. c,San Francisco,CA,94103,NaN,NaN,NaN,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk
4,sweetheart cafe,909 grant ave,San Francisco,CA,94108,NaN,NaN,NaN,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk
5,cafe new honolulu,888 stockton st,San Francisco,CA,94108,NaN,NaN,NaN,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk
6,aux delices vietnam,2327 polk st,San Francisco,CA,94109,37.798274,-122.422264,NaN,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk
7,cafe picaro,3120 16th st,San Francisco,CA,94103,37.764908,-122.422442,1.415543e+10,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk
8,jelly donut,3198 24th st,San Francisco,CA,94110,37.752439,-122.416250,NaN,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk
9,limon peruvian rotisserie,1001 south van ness ave,San Francisco,CA,94110,NaN,NaN,1.415555e+10,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk


In [27]:
def isNaN(x):
    return x != x

In [382]:
def block_on_postal_code(df):
    
    pcs = (df.business_postal_code.unique())
    #print(pcs)
    #pcs = ['94301', '94120', '95105', '94544', '94901', 'Ca', '00000', 'CA', '941', '94129', '941102019', '94080', '92672', '94188']
    for pc in pcs:
        pc_df = df[df['business_postal_code'] == pc]
        x = pc_df.drop_duplicates(subset=['business_name', 'business_address'])
        print("still alive", pc, pc_df.shape)
        run_loop_df(x)


In [383]:
def run_loop_old(df):
    for idx_r, r in df.iterrows():
        for idk_j, j in df.iterrows():
            #print(r.busines_name, j.business_name)
            s = get_score_df(r,j)
            if s:
                #print("MATCH!")
                #print(r['business_name'])
                #print(j['business_name'])
                #if r.business_name not in matches.keys() and j.business_name not in matches.keys():
                    #matches[r['business_name']] = j['business_name']
                if r.business_name not in moms.values() and j.business_name not in moms.values():
                    print("in here")
                    moms[idx_r] = r.business_name
                    synonyms[j.business_name] = idx_r
                    
                    


In [384]:
def run_loop_df(df):
    c = 0
    for r in df.itertuples():
        for j in df.itertuples():
            c += 1
            if get_first_address(i) == get_first_address(j):
                #print("stop gap caught it")
                s = get_score_df(r,j)
                if s:
                    #print(r.business_name)
                    #print(j.business_name)
                    print("c = ", c)
                    df = df.replace({r.business_name:j.business_name, r.business_address:j.business_address,
                         r.business_latitude:j.business_latitude, r.business_longitude:j.business_longitude,
                         r.business_phone_number:j.business_phone_number})

In [385]:
def get_score_df(i, j):
            
        #i = str(list(i.values)).split(",")
        i = list(i)
        i = i[1:]
        j = list(j)
        j = j[1:]
        #j = str(list(j.values)).split(",")


        b = jf.jaro_winkler(i[0], j[0])
        a = jf.jaro_winkler(i[1], j[1])
        pc = jf.jaro_winkler(str(i[4]), str(j[4]))
        
        if isNaN(i[7]) or isNaN(j[7]):
            #print("there is a nan")
            score = b*0.5 + a*0.45 + pc*0.05
        else:
            try:
                ph_n = jf.jaro_winkler(i[7], j[7])
                score = b*0.4 + a*0.4 + ph_n*0.15 + pc*0.05
            except:
                score = b*0.5 + a*0.45 + pc*0.05
            
        if score > 0.9:
            return True
        return False

In [386]:
def get_first_address(x):
        sp = x[1].strip().split(" ")
        sp2 = sp[0].strip("'")
        compare = len(sp2)
        return compare

In [387]:
small = rs.head(50000)

In [388]:
small

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,dar bar pakistani/indian cusine,1412 polk st,San Francisco,CA,94109,37.789784,-122.420455,NaN,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
1,dragon city bakery & cafe,2367 mission st,San Francisco,CA,94110,37.759174,-122.419066,1.415583e+10,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
2,deli 23,2449 23rd st,San Francisco,CA,94110,NaN,NaN,NaN,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
3,l'acajou bakery and cafe,498 09th st ste. c,San Francisco,CA,94103,NaN,NaN,NaN,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk
4,sweetheart cafe,909 grant ave,San Francisco,CA,94108,NaN,NaN,NaN,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk
5,cafe new honolulu,888 stockton st,San Francisco,CA,94108,NaN,NaN,NaN,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk
6,aux delices vietnam,2327 polk st,San Francisco,CA,94109,37.798274,-122.422264,NaN,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk
7,cafe picaro,3120 16th st,San Francisco,CA,94103,37.764908,-122.422442,1.415543e+10,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk
8,jelly donut,3198 24th st,San Francisco,CA,94110,37.752439,-122.416250,NaN,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk
9,limon peruvian rotisserie,1001 south van ness ave,San Francisco,CA,94110,NaN,NaN,1.415555e+10,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk


In [389]:
try:
    block_on_postal_code(small)
except:
    print(m, s)

still alive 94109 (3955, 15)
c =  2215
c =  2953
c =  7012
c =  7750
c =  11071
c =  11809
c =  13285
c =  14023
c =  17344
c =  23986
c =  25933
c =  29890
c =  31735
c =  35794
c =  41329
c =  42436
c =  42805
c =  43174
c =  46126
c =  46495
c =  47971
c =  49447
c =  50923
c =  51292
c =  52768
c =  53137
c =  53506
c =  53875
c =  54982
c =  59779
c =  60517
c =  60886
c =  63367
c =  63469
c =  63531
c =  72325
c =  73801
c =  74170
c =  78598
c =  79705
c =  80812
c =  84133
c =  86183
c =  86285
c =  86347
c =  88192
c =  89668
c =  91513
c =  92620
c =  92989
c =  98524
c =  99262
c =  102583
c =  111070
c =  111808
c =  112177
c =  115129
c =  120664
c =  122509
c =  127306
c =  134317
still alive 94110 (5704, 15)
c =  562
c =  3367
c =  3928
c =  7294
c =  11782
c =  12904
c =  23563
c =  25807
c =  26368
c =  34783
c =  42076
c =  57223
c =  61711
c =  65077
c =  71809
c =  72931
c =  73492
c =  88639
c =  93688
c =  94810
c =  105469
c =  109396
c =  116128
c =  129031
c =

c =  34945
c =  35329
c =  35713
still alive 94104 (970, 15)
c =  1
c =  273
c =  817
c =  1225
c =  1361
c =  3537
c =  4081
c =  4353
c =  4761
c =  5441
c =  7073
c =  7481
c =  8161
c =  8297
c =  8313
c =  8705
c =  8721
c =  8977
c =  9793
c =  9802
c =  10457
c =  10473
c =  10865
c =  10881
c =  11008
c =  11017
c =  11289
c =  14009
c =  15505
c =  16185
c =  18089
still alive 94123 (1575, 15)
c =  3421
c =  4618
c =  5473
c =  6499
c =  7354
c =  8722
c =  9577
c =  9748
c =  9919
c =  12484
c =  12826
c =  14878
c =  15049
c =  16930
c =  19666
c =  22231
c =  22744
c =  24283
c =  24454
c =  25138
c =  25480
c =  26164
c =  27361
c =  27703
still alive 94102 (3589, 15)
c =  1
c =  4841
c =  6161
c =  12761
c =  13201
c =  18921
c =  19801
c =  20681
c =  23321
c =  25521
c =  25961
c =  26841
c =  27281
c =  28601
c =  29041
c =  31241
c =  34321
c =  35201
c =  38281
c =  39161
c =  39193
c =  40041
c =  45761
c =  52801
c =  53060
c =  53209
c =  53241
c =  57641
c =  594

In [246]:
small

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,dar bar pakistani/indian cusine,1412 polk st,San Francisco,CA,94109,37.789784,-122.420455,NaN,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
1,dragon city bakery & cafe,2367 mission st,San Francisco,CA,94110,37.759174,-122.419066,1.415583e+10,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
2,deli 23,2449 23rd st,San Francisco,CA,94110,NaN,NaN,NaN,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
3,l'acajou bakery and cafe,498 09th st ste. c,San Francisco,CA,94103,NaN,NaN,NaN,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk
4,sweetheart cafe,909 grant ave,San Francisco,CA,94108,NaN,NaN,NaN,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk
5,cafe new honolulu,888 stockton st,San Francisco,CA,94108,NaN,NaN,NaN,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk
6,aux delices vietnam,2327 polk st,San Francisco,CA,94109,37.798274,-122.422264,NaN,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk
7,cafe picaro,3120 16th st,San Francisco,CA,94103,37.764908,-122.422442,1.415543e+10,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk
8,jelly donut,3198 24th st,San Francisco,CA,94110,37.752439,-122.416250,NaN,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk
9,limon peruvian rotisserie,1001 south van ness ave,San Francisco,CA,94110,NaN,NaN,1.415555e+10,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk


In [127]:
i = [0, 'dar bar pakistani/indian cusine', '1412 polk st', 'San Francisco', 'CA', '94109', 37.789784000000004, -122.420455, 'nan', '1757_20170928', '2017-09-28', 86.0, 'Routine - Unscheduled', '1757_20170928_103131', 'Moderate risk vermin infestation', 'Moderate Risk']


In [130]:
i = i[1:]
i

['dar bar pakistani/indian cusine',
 '1412 polk st',
 'San Francisco',
 'CA',
 '94109',
 37.789784000000004,
 -122.420455,
 'nan',
 '1757_20170928',
 '2017-09-28',
 86.0,
 'Routine - Unscheduled',
 '1757_20170928_103131',
 'Moderate risk vermin infestation',
 'Moderate Risk']

In [122]:
m

{0: 'dar bar pakistani/indian cusine',
 4: 'sweetheart cafe',
 6: 'aux delices vietnam',
 16: 'red a bakery',
 25: 'boudin restaurant',
 33: 'le marais bakery'}

In [45]:
s

{'boudin bakery & cafe': 25,
 'bow hon restaurant': 4,
 'flour craft bakery': 16,
 'modern thai inc.': 6,
 'super duper burgers': 33}

In [33]:
t = small.copy()

In [36]:
t = small.copy()

In [37]:
t

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,dar bar pakistani/indian cusine,1412 polk st,San Francisco,CA,94109,37.789784,-122.420455,NaN,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
1,dragon city bakery & cafe,2367 mission st,San Francisco,CA,94110,37.759174,-122.419066,1.415583e+10,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
2,deli 23,2449 23rd st,San Francisco,CA,94110,NaN,NaN,NaN,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
3,l'acajou bakery and cafe,498 09th st ste. c,San Francisco,CA,94103,NaN,NaN,NaN,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk
4,sweetheart cafe,909 grant ave,San Francisco,CA,94108,NaN,NaN,NaN,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk
5,cafe new honolulu,888 stockton st,San Francisco,CA,94108,NaN,NaN,NaN,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk
6,aux delices vietnam,2327 polk st,San Francisco,CA,94109,37.798274,-122.422264,NaN,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk
7,cafe picaro,3120 16th st,San Francisco,CA,94103,37.764908,-122.422442,1.415543e+10,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk
8,jelly donut,3198 24th st,San Francisco,CA,94110,37.752439,-122.416250,NaN,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk
9,limon peruvian rotisserie,1001 south van ness ave,San Francisco,CA,94110,NaN,NaN,1.415555e+10,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk


In [63]:
for idx, x in t.iterrows():
    i = str(list(x.values)).split(",")
    sp = i[1].strip().split(" ")
    sp2 = sp[0].strip("'")
    compare = len(sp2)
    print(sp2)

1412
2367
2449
498
909
888
2327
3120
3198
1001
876
off
24
595
2600
2399
634
1802
1131
2300
252
4019
634
726
1039
160
691
201
85
golden
200
2515
498
2066
2066
pier
pier
pier
1098
307
346
845
845
2999
pier
86
909
2512
595
1
4516
2232
500
550
1207
99
5176
inc.
655
2301
160
2444
1688
3251
142
142
2740
1941
132
151
845
1320
333
403
2626
3214
4798
65
2327
2800
100
932
901
813
81
church
960
24
llc
101
2527
inc.
298
2799
211
3100
673
pier
1532
3011


In [39]:
dups = list(s.values())
for idx, r in t.iterrows():
    if r.business_name in s.keys():
        link = s[r.business_name]
        ma = small.iloc[link]
        t = t.replace({r.business_name:ma.business_name, r.business_address:ma.business_address,
                     r.business_latitude:ma.business_latitude, r.business_longitude:ma.business_longitude,
                     r.business_phone_number:ma.business_phone_number})

In [50]:
x = t[t.business_name == 'terzo']

In [71]:
x[0]

'terzo'

In [51]:
i = str(list(x.values)).split(",")

In [56]:
sp = i[1].strip().split(" ")
sp

["'3011", 'steiner', "st'"]

In [61]:
sp[0].strip("'")

'3011'

In [41]:
def get_cleanrest(t):
    cleanrest = t.drop_duplicates(subset='business_name', keep='first', inplace=False)
    cleanrest = cleanrest[['business_name', 'business_address', 'business_city', 'business_state',
       'business_postal_code', 'business_latitude', 'business_longitude',
       'business_phone_number']]
    return cleanrest

In [42]:
cleanrest = get_cleanrest(t)

In [43]:
cleaninspection = t.copy()

In [48]:
cleaninspection.shape

(100, 15)